In [ ]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def scrape_npb_stats(ids):
    opts = Options()
    opts.add_argument("--headless=new")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--window-size=1920,1080")
    opts.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    )
    opts.add_experimental_option("excludeSwitches", ["enable-automation"])
    opts.add_experimental_option("useAutomationExtension", False)

    driver = webdriver.Chrome(options=opts)
    driver.execute_cdp_cmd(
        "Page.addScriptToEvaluateOnNewDocument",
        {
            "source": (
                "Object.defineProperty(navigator, 'webdriver', "
                "{get: () => undefined})"
            )
        },
    )

    wait = WebDriverWait(driver, 20)
    all_records = []
    out_rows = []
    
    try:
        for pid in ids:
            url = f"http://npbstats.com/players/db/batting/?wdt_search={pid}"
            driver.get(url)

            wait.until(
                EC.presence_of_element_located(
                    (
                        By.CSS_SELECTOR,
                        "#table_47 tbody tr.odd, #table_47 tbody tr.even",
                    )
                )
            )
            time.sleep(0.2)

            soup = BeautifulSoup(driver.page_source, "lxml")

            # table_1 bio
            bio_tbl = soup.find("table", id="table_1")
            bio_df = pd.read_html(str(bio_tbl), flavor="lxml")[0]
            bio_df.columns = bio_df.columns.str.strip()
            bio = bio_df.iloc[0].to_dict()
            bio["SearchID"] = pid

            # table_47 standard 
            fid_tbl = soup.find("table", id="table_47")
            fid_df = pd.read_html(str(fid_tbl), flavor="lxml")[0]
            fid_df.columns = fid_df.columns.str.strip()
            fid_df["SearchID"] = pid

            for _, season in fid_df.iterrows():
                out_rows.append({**bio, **season.to_dict()})
    finally:
        driver.quit()

    return pd.DataFrame(out_rows)


if __name__ == "__main__":
    ids = [str(pid) for pid in range(100001, 100003)]
    df = scrape_npb_stats(ids)

    print(df.head(20).to_string(index=False))

    print("rows:", len(df), "| columns:", len(df.columns))
    print("columns:", list(df.columns))

           Player      Birthplace Height/Weight Bats/Throws      Position  Brithdate SearchID    Year  Lg     Tm Pos  G    PO    A   E   DP  PB  WP    FP   RF  RRF  rRng  rErr  rWP  rPB  rCth  rDP  rArm   FS
Eiichiro Yamamoto  Shimane, Japan  157cm / 58kg Right/Right Right fielder 03/08/1902   100001 1936.1S JBL Kyojin  2B  2   NaN  NaN NaN  NaN NaN NaN   NaN  NaN  0.0   0.0   0.0  NaN  NaN   NaN  0.0   NaN  0.0
Eiichiro Yamamoto  Shimane, Japan  157cm / 58kg Right/Right Right fielder 03/08/1902   100001 1936.1S JBL Kyojin  OF  2   NaN  NaN NaN  NaN NaN NaN   NaN  NaN  0.0   0.0   0.0  NaN  NaN   NaN  NaN   0.0  0.0
Eiichiro Yamamoto  Shimane, Japan  157cm / 58kg Right/Right Right fielder 03/08/1902   100001 1936.2F JBL Kyojin  OF 14  13.0  2.0 1.0  1.0 NaN NaN 0.938 1.07 -0.5  -0.1  -0.1  NaN  NaN   NaN  NaN   1.1  0.9
Eiichiro Yamamoto  Shimane, Japan  157cm / 58kg Right/Right Right fielder 03/08/1902   100001 1937.1S JBL Kyojin  OF 14   4.0  0.0 0.0  0.0 NaN NaN 1.000 0.29 -1.8  -0.

In [ ]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from tqdm import tqdm

def safe_get_and_wait(driver, wait, url, max_retries=2):
    """Load `url` and wait for table 47 to appear; retry on TimeoutException."""
    for attempt in range(max_retries + 1):
        try:
            driver.get(url)
            wait.until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "#table_47 tbody tr.odd, #table_47 tbody tr.even")
                )
            )
            time.sleep(0.2)
            return True
        except TimeoutException:
            if attempt < max_retries:
                backoff = 2 ** attempt
                print(f"[warn] timeout {url} — retry {attempt+1} in {backoff}s")
                time.sleep(backoff)
            else:
                print(f"[error] giving up on {url}")
                return False

def scrape_npb_stats(ids, chunk_size=5000):
    opts = Options()
    opts.add_argument("--headless=new")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--window-size=1920,1080")
    opts.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)")
    opts.add_experimental_option("excludeSwitches", ["enable-automation"])
    opts.add_experimental_option("useAutomationExtension", False)

    driver = webdriver.Chrome(options=opts)
    driver.execute_cdp_cmd(
        "Page.addScriptToEvaluateOnNewDocument",
        {"source": "Object.defineProperty(navigator,'webdriver',{get:()=>undefined})"},
    )
    wait = WebDriverWait(driver, 20)

    out_rows, chunk_idx = [], 1
    try:
        pbar = tqdm(ids, desc="Scraping fielding")
        for pid in pbar:
            pbar.set_postfix(pid=pid)

            url = f"http://npbstats.com/players/db/batting/?wdt_search={pid}"
            if not safe_get_and_wait(driver, wait, url):
                continue

            soup = BeautifulSoup(driver.page_source, "lxml")

            # table 1
            bio_tbl = soup.find("table", id="table_1")
            if not bio_tbl or bio_tbl.find("td", class_="dataTables_empty"):
                continue
            bio_df = pd.read_html(str(bio_tbl), flavor="lxml")[0]
            bio_df.columns = bio_df.columns.str.strip()
            bio = bio_df.iloc[0].to_dict()
            bio["SearchID"] = pid

            # table 47
            fld_tbl = soup.find("table", id="table_47")
            fld_df = pd.read_html(str(fld_tbl), flavor="lxml")[0]
            fld_df.columns = fld_df.columns.str.strip()
            fld_df["SearchID"] = pid
            fld_df = fld_df[fld_df["Year"] != "データはありません。"]
            if fld_df.empty:
                continue

            for _, season in fld_df.iterrows():
                out_rows.append({**bio, **season.to_dict()})

            if len(out_rows) >= chunk_size:
                df_chunk = pd.DataFrame(out_rows)
                fname = f"npbstats_fielding_chunk_{chunk_idx}.csv"
                df_chunk.to_csv(fname, index=False)
                pbar.write(f"[saved] {fname}")
                chunk_idx += 1
                out_rows.clear()

    finally:
        driver.quit()

    return pd.DataFrame(out_rows)


if __name__ == "__main__":
    ids = [str(pid) for pid in range(100001, 107185)]
    df_remain = scrape_npb_stats(ids)

    if not df_remain.empty:
        df_remain.to_csv("npbstats_fielding_final.csv", index=False)

Scraping fielding:   6%|▋         | 460/7184 [2:35:39<37:24:09, 20.03s/it, pid=100461]

[saved] npbstats_fielding_chunk_1.csv


Scraping fielding:  15%|█▍        | 1058/7184 [5:41:31<36:55:54, 21.70s/it, pid=101059]

[saved] npbstats_fielding_chunk_2.csv


Scraping fielding:  18%|█▊        | 1260/7184 [6:40:54<36:44:43, 22.33s/it, pid=101261]

...

[warn] timeout http://npbstats.com/players/db/batting/?wdt_search=105883 — retry 1 in 1s


Scraping fielding:  91%|█████████ | 6529/7184 [32:09:14<1:42:31,  9.39s/it, pid=106530] 

[saved] npbstats_fielding_chunk_13.csv


Scraping fielding: 100%|██████████| 7184/7184 [34:00:32<00:00, 17.04s/it, pid=107184]  


Saved npbstats_fielding_final.csv
Done scraping.


In [ ]:
import glob
import pandas as pd
from pathlib import Path

csv_files = sorted(glob.glob("npbstats_fielding_chunk_*.csv") + ["npbstats_fielding_final.csv"], key=lambda p: (p.count("_"), p))

Found 14 files: ['npbstats_fielding_final.csv', 'npbstats_fielding_chunk_1.csv', 'npbstats_fielding_chunk_10.csv', 'npbstats_fielding_chunk_11.csv', 'npbstats_fielding_chunk_12.csv', 'npbstats_fielding_chunk_13.csv', 'npbstats_fielding_chunk_2.csv', 'npbstats_fielding_chunk_3.csv', 'npbstats_fielding_chunk_4.csv', 'npbstats_fielding_chunk_5.csv', 'npbstats_fielding_chunk_6.csv', 'npbstats_fielding_chunk_7.csv', 'npbstats_fielding_chunk_8.csv', 'npbstats_fielding_chunk_9.csv']


In [ ]:
frames = []
for f in csv_files:
    df = pd.read_csv(f)
    frames.append(df)

In [ ]:
big_df = pd.concat(frames, ignore_index=True)
big_df = big_df.sort_values("SearchID", key=lambda s: s.astype(int))  # ← sort

out_path = Path("npbstats_fielding_full.csv")
big_df.to_csv(out_path, index=False